# MissionWired Hiring
### Data Engineer Exercise

The purpose of this exercise is to evaluate your level of skill when it comes to manipulating and aggregating a large dataset through code. We’ll evaluate the quality, output and readability of your code as well as the efficacy of provided documentation.

We recommend using Python and Pandas to complete this exercise. Most of our production data engineering work is done using Python, Pandas and PySpark (a “big data” alternative to Pandas).

We recommend submitting your code by way of a personal GitHub repository. Directly submitting code files is also acceptable.

Draft documentation describing how a reviewer can run your app locally. Be sure to include steps like installing dependencies or other “pre-flight” configurations necessary for your code to run.

## Dataset

A dataset simulating CRM data is available in some public AWS S3 files:

* Constituent Information: https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons.csv
* Constituent Email Addresses: https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email.csv
    - Boolean columns (including is_primary) in all of these datasets are 1/0 numeric values. 1 means True, 0 means False.
* Constituent Subscription Status: https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email_chapter_subscription.csv
    - We only care about subscription statuses where chapter_id is 1.
    - If an email is not present in this table, it is assumed to still be subscribed where chapter_id is 1.

Use these files to complete the exercises below.


# 0. Load Data
Three dataframes were created from the data coming from the three .CSV files

In [20]:
import pandas as pd
cons = pd.read_csv('https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons.csv')
cons_email = pd.read_csv('https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email.csv')
cons_email_chapter_subscription = pd.read_csv('https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email_chapter_subscription.csv')

# uncomment if you download the csv files to the working directory
# cons = pd.read_csv('cons.csv')
# cons_email = pd.read_csv('cons_email.csv')
# cons_email_chapter_subscription = pd.read_csv('cons_email_chapter_subscription.csv')

Dataframes were explored

In [21]:
# Constituent Information
print(cons.shape)
cons.head(10)

(700000, 29)


,cons_id,prefix,firstname,middlename,lastname,suffix,salutation,gender,birth_dt,title,...,change_password_next_login,consent_type_id,create_dt,create_app,create_user,modified_dt,modified_app,modified_user,status,note
0,1,NaN,NaN,Lee,NaN,MD,NaN,E,NaN,vSkSIzEQJdXnqeTTTXSG,...,0,5958,"Fri, 1983-08-26 06:02:03",1484,6162,"Sun, 2015-12-27 09:28:02",4022,6349,1,NaN
1,2,NaN,NaN,NaN,NaN,II,boFqBKgLlSgEZsFrgCZd,E,"Mon, 2004-11-15",NaN,...,1,4236,"Mon, 1979-03-05 21:08:54",4176,5476,"Tue, 1989-06-20 13:28:57",9010,5698,1,NaN
2,3,NaN,NaN,David,King,NaN,NaN,D,"Fri, 1994-04-08",bxGxufoNzpKvjwNIxgRj,...,1,1263,"Fri, 2008-08-22 19:20:28",4702,8239,"Fri, 2020-06-05 18:13:57",8837,1175,1,NaN
3,4,Ms.,Mark,Gregg,NaN,NaN,NaN,J,NaN,NaN,...,0,949,"Sun, 1984-04-29 11:18:18",7096,7875,"Tue, 2012-01-31 07:26:35",7529,3984,1,NaN
4,5,NaN,NaN,Logan,NaN,DDS,ArvPJuEozHPaqbirovMi,D,"Sun, 2008-08-10",kXKlKCyCBoXUBVejPOVO,...,0,7126,"Thu, 2007-07-19 18:28:09",706,5727,"Tue, 1971-04-27 06:53:53",9356,6062,1,NaN
5,6,NaN,NaN,NaN,Holt,NaN,NaN,K,"Tue, 2003-07-15",DBEfLmqhahjEXOXjLHpF,...,1,9079,"Sat, 2014-02-15 22:54:27",7328,6425,"Fri, 1999-11-12 06:09:25",2118,2494,0,productize customized e-services
6,7,Mr.,Alexandra,Brian,Baldwin,DVM,NaN,X,NaN,NaN,...,1,5809,"Thu, 1979-03-01 16:12:55",9610,6785,"Wed, 2019-03-13 02:52:10",1680,1864,1,NaN
7,8,NaN,Charles,Teresa,Glover,PhD,NaN,O,"Mon, 1977-04-04",NaN,...,0,1416,"Tue, 2002-07-30 22:28:38",8257,542,"Sat, 1983-07-16 18:30:07",1429,3195,1,evolve real-time relationships
8,9,NaN,Holly,David,NaN,NaN,NaN,K,NaN,NaN,...,0,5614,"Thu, 2019-08-29 14:50:24",4561,1718,"Sun, 2009-12-20 14:50:14",3368,1975,1,NaN
9,10,NaN,Chad,NaN,NaN,DVM,YupmMCKEGDZOxnqeIjha,NaN,NaN,NaN,...,0,7669,"Tue, 1987-02-17 11:22:04",8761,1288,"Sat, 2016-03-19 17:33:38",1405,5181,1,NaN


In [22]:
#  Constituent Email Addresses
print(cons_email.shape)
cons_email.head(10)

(1400000, 16)


,cons_email_id,cons_id,cons_email_type_id,is_primary,email,canonical_local_part,domain,double_validation,create_dt,create_app,create_user,modified_dt,modified_app,modified_user,status,note
0,1,548198,3361,1,xmartinez@vincent.com,NaN,gmail.com,NaN,"Wed, 1994-01-26 23:49:16",4072,9954,"Sat, 2014-04-19 19:10:39",1990,7595,1,NaN
1,2,491137,2474,1,hmiller@haynes.biz,jqCyozTDojYuylQPTHfm,hotmail.com,NaN,"Thu, 1999-12-09 06:18:27",1600,5716,"Sat, 1984-07-14 05:55:27",4686,3248,1,NaN
2,3,413429,5175,1,aaron64@yahoo.com,FCBeBiVoqnnKDWjnllhN,yahoo.com,kRLZexQEwYIMbwFNvQxg,"Wed, 1992-11-18 16:46:27",7358,2685,"Sun, 1995-12-24 13:13:01",3857,7405,1,NaN
3,4,347346,4117,1,wyattvincent@hotmail.com,NaN,gmail.com,zSbfmlqXimGyWVBUGdQg,"Sat, 1983-11-26 16:49:14",881,3444,"Sun, 1975-01-19 14:32:56",8713,7713,1,NaN
4,5,443000,6781,1,tspencer@hotmail.com,VaQIYlKcUkIywkKKEptD,gmail.com,NaN,"Wed, 2000-11-15 13:28:34",5380,5456,"Sun, 1994-03-13 16:38:37",765,8618,1,NaN
5,6,412213,226,1,ogarcia@gmail.com,plEoepTGqWOhdZUZsEKK,gmail.com,NaN,"Thu, 1984-03-08 11:40:21",3423,4910,"Mon, 1986-04-07 08:14:46",99,4832,1,NaN
6,7,282118,5477,1,scott06@thompson.info,txFZqAuoDzXhrrKJePGJ,yahoo.com,NaN,"Tue, 1975-11-25 06:42:33",8112,252,"Mon, 1975-06-02 18:42:40",1924,9055,1,NaN
7,8,624574,5784,1,allenrobert@yahoo.com,tjaiAMmxqpBKiTgKvcfN,gmail.com,NaN,"Sun, 1973-11-11 20:10:53",1901,6427,"Mon, 1995-07-10 03:12:20",2544,6526,1,NaN
8,9,603436,7790,1,madeline69@mccarthy-jackson.com,NaN,hotmail.com,TTSshhSpgVPLzlrEBkjo,"Mon, 1976-06-28 08:58:19",7418,8577,"Tue, 2001-07-24 12:10:57",741,4342,1,NaN
9,10,185213,5912,1,bishopshannon@gmail.com,NaN,gmail.com,FAjeAoTZTNAikikCualt,"Sun, 1988-08-07 10:51:50",960,1269,"Sat, 2013-06-22 23:28:06",1619,1698,1,NaN


In [23]:
# Constituent Subscription Status:
print(cons_email_chapter_subscription.shape)
cons_email_chapter_subscription.head(10)

(350000, 6)


,cons_email_chapter_subscription_id,cons_email_id,chapter_id,isunsub,unsub_dt,modified_dt
0,1,332188,1,1,"Sat, 1971-06-12 15:38:44","Thu, 1990-06-28 10:54:20"
1,2,536526,1,1,"Wed, 2006-07-12 01:50:45","Thu, 1979-09-20 06:02:35"
2,3,134711,1,1,"Tue, 1987-01-06 13:05:15","Sun, 1974-03-03 15:11:50"
3,4,660345,1,1,"Sat, 2016-08-06 11:06:09","Wed, 1995-09-13 23:45:03"
4,5,184268,1,1,"Sun, 2000-05-28 02:20:45","Sat, 1983-12-10 08:09:58"
5,6,621735,1,1,"Wed, 2009-12-09 14:08:54","Sat, 1990-09-01 06:53:49"
6,7,449159,1,1,"Fri, 2001-02-16 03:25:34","Fri, 2018-04-27 17:26:03"
7,8,684697,1,1,"Wed, 1988-06-29 09:32:53","Mon, 1978-06-12 17:50:09"
8,9,247669,1,1,"Tue, 2003-07-08 23:52:27","Wed, 1972-08-09 06:43:57"
9,10,298884,1,1,"Sat, 1986-09-20 16:39:10","Wed, 1974-09-25 07:52:08"


# Exercise 1

Produce a “people” file with the following schema. Save it as a CSV with a header line to the working directory.

<img src="schema1.png" alt="Schema Exercise 1" />

To do this I merge Constituent Information (cons) with Constituent Email Addresses (cons_email) using the key cons_id with an inner join. I have also only selected the records from cons_email where is_primary equals 1.

With this I created a first version of people dataframe with the columns source code, created date, updated date and primary email.

In [24]:
# alternative 1 - inner join
people = pd.merge(cons[['cons_id', 'source','create_dt', 'modified_dt']], cons_email[cons_email.is_primary == 1][['cons_id', 'email', 'cons_email_id']], how='inner', on = 'cons_id')
print(people.shape)
people.head(10)

# alternative 2 - left join (to get all records from cons and the matches in cons_email) I have not selected this option because it generates records without email, something that should be defined with the stakeholders.
# people = pd.merge(cons[['cons_id', 'source','create_dt', 'modified_dt']], cons_email[cons_email.is_primary == 1][['cons_id', 'email', 'cons_email_id']], how='left', on = 'cons_id')
# print(people.shape)
# people.head(10)

(605639, 6)


,cons_id,source,create_dt,modified_dt,email,cons_email_id
0,1,google,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02",daniel72@hudson.com,546912
1,2,facebook,"Mon, 1979-03-05 21:08:54","Tue, 1989-06-20 13:28:57",caustin@spears-carson.com,415587
2,3,NaN,"Fri, 2008-08-22 19:20:28","Fri, 2020-06-05 18:13:57",klewis@ford.biz,81594
3,5,NaN,"Thu, 2007-07-19 18:28:09","Tue, 1971-04-27 06:53:53",stephenhamilton@gmail.com,182741
4,6,google,"Sat, 2014-02-15 22:54:27","Fri, 1999-11-12 06:09:25",henryherring@gmail.com,246591
5,7,NaN,"Thu, 1979-03-01 16:12:55","Wed, 2019-03-13 02:52:10",perrymatthew@campbell.com,339289
6,8,twitter,"Tue, 2002-07-30 22:28:38","Sat, 1983-07-16 18:30:07",tonifranklin@yahoo.com,1057700
7,10,twitter,"Tue, 1987-02-17 11:22:04","Sat, 2016-03-19 17:33:38",gillderrick@melton.net,1398350
8,11,google,"Thu, 2018-04-19 10:44:52","Fri, 1993-06-25 22:34:12",tomnelson@gmail.com,1324937
9,12,organic,"Sun, 2004-04-11 18:52:49","Mon, 2018-07-16 00:25:16",trevinomary@gmail.com,675752


Because the document contains the requirement "We only care about subscription statuses where chapter_id is 1". I have created a new dataframe of cons_email_chapter_subscription selecting only the records with chapter_id = 1 and the columns cons_email_id and isunsub for a future merge.

In [25]:
cons_email_chapter_subscription_chapter_1 = cons_email_chapter_subscription[cons_email_chapter_subscription.chapter_id == 1][['cons_email_id', 'isunsub']]
print(cons_email_chapter_subscription_chapter_1.shape)
cons_email_chapter_subscription_chapter_1.head()

(275484, 2)


,cons_email_id,isunsub
0,332188,1
1,536526,1
2,134711,1
3,660345,1
4,184268,1


To add the isunsub column to the people dataframe I have performed a merge using the cons_email_id key with a left join (to get all records from cons and the matches in cons_email_chapter_subscription_chapter_1)

In [26]:
# add is_unsub to people dataframe
people = pd.merge(people, cons_email_chapter_subscription_chapter_1, how='left', on='cons_email_id')
print(people.shape)
people.head(15)

(605639, 7)


,cons_id,source,create_dt,modified_dt,email,cons_email_id,isunsub
0,1,google,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02",daniel72@hudson.com,546912,NaN
1,2,facebook,"Mon, 1979-03-05 21:08:54","Tue, 1989-06-20 13:28:57",caustin@spears-carson.com,415587,1.0
2,3,NaN,"Fri, 2008-08-22 19:20:28","Fri, 2020-06-05 18:13:57",klewis@ford.biz,81594,1.0
3,5,NaN,"Thu, 2007-07-19 18:28:09","Tue, 1971-04-27 06:53:53",stephenhamilton@gmail.com,182741,1.0
4,6,google,"Sat, 2014-02-15 22:54:27","Fri, 1999-11-12 06:09:25",henryherring@gmail.com,246591,NaN
5,7,NaN,"Thu, 1979-03-01 16:12:55","Wed, 2019-03-13 02:52:10",perrymatthew@campbell.com,339289,NaN
6,8,twitter,"Tue, 2002-07-30 22:28:38","Sat, 1983-07-16 18:30:07",tonifranklin@yahoo.com,1057700,NaN
7,10,twitter,"Tue, 1987-02-17 11:22:04","Sat, 2016-03-19 17:33:38",gillderrick@melton.net,1398350,NaN
8,11,google,"Thu, 2018-04-19 10:44:52","Fri, 1993-06-25 22:34:12",tomnelson@gmail.com,1324937,NaN
9,12,organic,"Sun, 2004-04-11 18:52:49","Mon, 2018-07-16 00:25:16",trevinomary@gmail.com,675752,NaN


Due to the requirement in the document (If an email is not present in this table, it is assumed to still be subscribed where chapter_id is 1.) I have updated all the null values to 0 (false) of the insunsub column.

In [27]:
people['isunsub'] = people['isunsub'].fillna(0)
print(people.shape)
people.head(15)

(605639, 7)


,cons_id,source,create_dt,modified_dt,email,cons_email_id,isunsub
0,1,google,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02",daniel72@hudson.com,546912,0.0
1,2,facebook,"Mon, 1979-03-05 21:08:54","Tue, 1989-06-20 13:28:57",caustin@spears-carson.com,415587,1.0
2,3,NaN,"Fri, 2008-08-22 19:20:28","Fri, 2020-06-05 18:13:57",klewis@ford.biz,81594,1.0
3,5,NaN,"Thu, 2007-07-19 18:28:09","Tue, 1971-04-27 06:53:53",stephenhamilton@gmail.com,182741,1.0
4,6,google,"Sat, 2014-02-15 22:54:27","Fri, 1999-11-12 06:09:25",henryherring@gmail.com,246591,0.0
5,7,NaN,"Thu, 1979-03-01 16:12:55","Wed, 2019-03-13 02:52:10",perrymatthew@campbell.com,339289,0.0
6,8,twitter,"Tue, 2002-07-30 22:28:38","Sat, 1983-07-16 18:30:07",tonifranklin@yahoo.com,1057700,0.0
7,10,twitter,"Tue, 1987-02-17 11:22:04","Sat, 2016-03-19 17:33:38",gillderrick@melton.net,1398350,0.0
8,11,google,"Thu, 2018-04-19 10:44:52","Fri, 1993-06-25 22:34:12",tomnelson@gmail.com,1324937,0.0
9,12,organic,"Sun, 2004-04-11 18:52:49","Mon, 2018-07-16 00:25:16",trevinomary@gmail.com,675752,0.0


Next I have eliminated the unnecessary columns 'cons_id' and 'cons_email_id'

In [28]:
people = people.drop(columns=['cons_id', 'cons_email_id'])
print(people.shape)
people.head(15)

(605639, 5)


,source,create_dt,modified_dt,email,isunsub
0,google,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02",daniel72@hudson.com,0.0
1,facebook,"Mon, 1979-03-05 21:08:54","Tue, 1989-06-20 13:28:57",caustin@spears-carson.com,1.0
2,NaN,"Fri, 2008-08-22 19:20:28","Fri, 2020-06-05 18:13:57",klewis@ford.biz,1.0
3,NaN,"Thu, 2007-07-19 18:28:09","Tue, 1971-04-27 06:53:53",stephenhamilton@gmail.com,1.0
4,google,"Sat, 2014-02-15 22:54:27","Fri, 1999-11-12 06:09:25",henryherring@gmail.com,0.0
5,NaN,"Thu, 1979-03-01 16:12:55","Wed, 2019-03-13 02:52:10",perrymatthew@campbell.com,0.0
6,twitter,"Tue, 2002-07-30 22:28:38","Sat, 1983-07-16 18:30:07",tonifranklin@yahoo.com,0.0
7,twitter,"Tue, 1987-02-17 11:22:04","Sat, 2016-03-19 17:33:38",gillderrick@melton.net,0.0
8,google,"Thu, 2018-04-19 10:44:52","Fri, 1993-06-25 22:34:12",tomnelson@gmail.com,0.0
9,organic,"Sun, 2004-04-11 18:52:49","Mon, 2018-07-16 00:25:16",trevinomary@gmail.com,0.0


Because the schema requires a specific data type for the columns I have checked the data types and updated according to the schema

In [29]:
# check columns data types
people.dtypes

source          object
create_dt       object
modified_dt     object
email           object
isunsub        float64
dtype: object

In [30]:
# convert column data types
people = people.convert_dtypes()
people['create_dt'] = pd.to_datetime(people['create_dt'])
people['modified_dt'] = pd.to_datetime(people['modified_dt'])
people['isunsub'] = people['isunsub'].astype('bool')
print(people.dtypes)
print(people.shape)
people.head(15)

source                 string
create_dt      datetime64[ns]
modified_dt    datetime64[ns]
email                  string
isunsub                  bool
dtype: object
(605639, 5)


,source,create_dt,modified_dt,email,isunsub
0,google,1983-08-26 06:02:03,2015-12-27 09:28:02,daniel72@hudson.com,False
1,facebook,1979-03-05 21:08:54,1989-06-20 13:28:57,caustin@spears-carson.com,True
2,<NA>,2008-08-22 19:20:28,2020-06-05 18:13:57,klewis@ford.biz,True
3,<NA>,2007-07-19 18:28:09,1971-04-27 06:53:53,stephenhamilton@gmail.com,True
4,google,2014-02-15 22:54:27,1999-11-12 06:09:25,henryherring@gmail.com,False
5,<NA>,1979-03-01 16:12:55,2019-03-13 02:52:10,perrymatthew@campbell.com,False
6,twitter,2002-07-30 22:28:38,1983-07-16 18:30:07,tonifranklin@yahoo.com,False
7,twitter,1987-02-17 11:22:04,2016-03-19 17:33:38,gillderrick@melton.net,False
8,google,2018-04-19 10:44:52,1993-06-25 22:34:12,tomnelson@gmail.com,False
9,organic,2004-04-11 18:52:49,2018-07-16 00:25:16,trevinomary@gmail.com,False


Next, I have renamed and rearranged the columns according to the schema

In [31]:
# rename columns according to schema
people = people.rename(columns={'source': 'code', 'isunsub': 'is_unsub', 'modified_dt': 'updated_dt'})
people.head()

,code,create_dt,updated_dt,email,is_unsub
0,google,1983-08-26 06:02:03,2015-12-27 09:28:02,daniel72@hudson.com,False
1,facebook,1979-03-05 21:08:54,1989-06-20 13:28:57,caustin@spears-carson.com,True
2,<NA>,2008-08-22 19:20:28,2020-06-05 18:13:57,klewis@ford.biz,True
3,<NA>,2007-07-19 18:28:09,1971-04-27 06:53:53,stephenhamilton@gmail.com,True
4,google,2014-02-15 22:54:27,1999-11-12 06:09:25,henryherring@gmail.com,False


In [32]:
# rearranged columns according to schema
people = people[['email', 'code', 'is_unsub', 'create_dt', 'updated_dt']]
print(people.shape)
people

(605639, 5)


,email,code,is_unsub,create_dt,updated_dt
0,daniel72@hudson.com,google,False,1983-08-26 06:02:03,2015-12-27 09:28:02
1,caustin@spears-carson.com,facebook,True,1979-03-05 21:08:54,1989-06-20 13:28:57
2,klewis@ford.biz,<NA>,True,2008-08-22 19:20:28,2020-06-05 18:13:57
3,stephenhamilton@gmail.com,<NA>,True,2007-07-19 18:28:09,1971-04-27 06:53:53
4,henryherring@gmail.com,google,False,2014-02-15 22:54:27,1999-11-12 06:09:25
...,...,...,...,...,...
605634,cobbpatricia@anderson.biz,<NA>,False,2008-06-16 18:49:16,1999-10-23 22:19:10
605635,hjones@raymond-adkins.com,<NA>,True,2001-11-15 08:04:01,2010-08-24 07:06:01
605636,jennifer82@oneal.com,twitter,False,1973-06-14 08:14:54,1995-09-29 17:28:40
605637,robertodixon@wilson.net,google,False,1998-06-02 01:24:00,2014-06-01 11:16:45


Finally, the dataframe is ready to be exported as a .CSV file

In [33]:
# export to csv file
people.to_csv('people.csv')

# Exercise 2

Use the output of #1 to produce an “acquisition_facts” file with the following schema that aggregates stats about when people in the dataset were acquired. Save it to the working directory.

<img src="schema2.png" alt="Schema Exercise 2" />

To do this I have used the people dataframe grouping by days using the create_dt column and count using the email column.

In [34]:
acquisition_facts = people.groupby([people['create_dt'].dt.date])['email'].count().reset_index()
# rename columns
acquisition_facts = acquisition_facts.rename(columns={'create_dt': 'acquisition_date', 'email': 'acquisitions'})
print(acquisition_facts.shape)
acquisition_facts.head(15)

(18445, 2)


,acquisition_date,acquisitions
0,1970-01-01,28
1,1970-01-02,34
2,1970-01-03,27
3,1970-01-04,36
4,1970-01-05,39
5,1970-01-06,33
6,1970-01-07,33
7,1970-01-08,28
8,1970-01-09,35
9,1970-01-10,36


The dataframe is ready to be exported as a .CSV file

In [35]:
acquisition_facts.to_csv('acquisition_facts.csv')

# Client Communications

Consider the work you just completed in the Data Engineer Exercise. Please write no more than 2 paragraphs explaining to a client how you approached this task. Remember that clients do not need to know every process detail, but do want to understand how and why your choices contribute to our overall strategy and any benefits of the end product.


Thank you for the opportunity to work with you. According to the request received, three datasets were received, one from Constituent Information with 700,000 records and 29 columns, another from Constituent Email Addresses with 1,400,000 records and 16 columns and finally another with Constituent Subscription Status with 350,000 records and 6 columns. To answer the first question, We started with a merge between Constituent Information and Constituent Email Addresses using the cons_id key and using an inner join to guarantee all people with email. We have also only selected the records from cons_email where is_primary equals 1 according to requirement. With the above, a first version of the people dataset was obtained with the columns source, create_dt, modified_dt and e_mail in addition to the keys that were used to make the joins. With this result, a second merge was performed with the Constituent Subscription Status data using the cons_email_id key and a left join with which the isunsub column was added. We also filtered the dataset where chapter_id is 1 since we were given a requirement to remove all other chapte_ids in Constituent Subscription Status.

Finally, the null values in the isunsub column of the result were changed to 0 according to the requirement (If an email is not present in this table, it is assumed to still be subscribed where chapter_id is 1). With the above, the result had the necessary columns, so We proceeded to eliminate the unnecessary columns (cons_id and cons_email_id). Next, the type of data and the name of the columns were also verified, and they were changed according to the schema provided. Finally, the result with 605,639 records and six columns was exported to a .CSV file. For the second exercise, the previous results were grouped by days using the create_dt column and count using the email column to obtain the number of acquisitions for each day. Then the columns were renamed according to the scheme. Finally, the result with 18,445 rows and two columns was exported to a .CSV file.